In [1]:
import numpy as np
import torch
from tsGaussian.torch_tsgaussian import TangentSpaceGaussian
from stable_baselines_utils import TangentSpaceGaussian as TSG

In [2]:
tg = TangentSpaceGaussian(None)

# Test liegroup torch

In [3]:
from liegroups.torch import SO3

In [4]:
C = SO3.exp(torch.Tensor([[1,2,3],
                         [0,0,0]]))
C

<liegroups.torch.so3.SO3Matrix>
| tensor([[[-0.6949,  0.7135,  0.0893],
|          [-0.1920, -0.3038,  0.9332],
|          [ 0.6930,  0.6313,  0.3481]],
| 
|         [[ 1.0000,  0.0000,  0.0000],
|          [ 0.0000,  1.0000,  0.0000],
|          [ 0.0000,  0.0000,  1.0000]]])

# Test torch_tsgaussian sample

In [5]:
R_mu = torch.eye(3).reshape((1,3,3))
sigma = torch.ones(3).reshape((1,3))

In [6]:
tg.rsample(R_mu, sigma)

omega size:  torch.Size([1, 3])


tensor([[[-0.2935, -0.8591,  0.4193],
         [ 0.8864, -0.0804,  0.4558],
         [-0.3579,  0.5055,  0.7851]]])

# Test torch_tsgaussian normal_term

In [7]:
sigma = torch.ones(3).reshape((1,3))
sigma

tensor([[1., 1., 1.]])

In [8]:
tg.normal_term(sigma)

tensor([15.7496])

# Test torch_tsgaussian log_map

In [9]:
R_1 = torch.eye(3).reshape((1, 3, 3))
R_2 = torch.eye(3).reshape((1, 3, 3))

In [10]:
tg.log_map(R_1, R_2)

tensor([0., 0., 0.])

# Test torch_tsgaussian log_probs

In [11]:
R_x = torch.eye(3).reshape((1,3,3))
R_mu = torch.zeros(3,3).reshape((1,3,3))
R_x = R_x.repeat(5, 1, 1)
R_mu = R_mu.repeat(5, 1, 1)
sigma = torch.ones(3).reshape((1,3))

In [12]:
tg.log_probs(R_x, R_mu, sigma)

tensor([-2.7568, -2.7568, -2.7568, -2.7568, -2.7568])

In [13]:
np.e ** (-2.7568)

0.06349462641817973

all codes run for torch_tsgaussian now, need to check it's correctness and make it into batch version.

# Test TangentSpaceGaussian actions_from_params

In [14]:
tsg = TSG(None)

In [15]:
print(tsg.distribution)

In [16]:
tsg

In [17]:
tsg.actions_from_params(torch.eye(3).reshape((1,3,3)), torch.ones(3).reshape((1,3)))

self.sigma size:  torch.Size([1, 3])
omega size:  torch.Size([1, 3])
actions size:  torch.Size([1, 3, 3])
actions type:  <class 'torch.Tensor'>
self.sigma size:  torch.Size([1, 3])
omega size:  torch.Size([1, 3])


tensor([[[ 0.3730, -0.9207, -0.1145],
         [-0.4982, -0.0946, -0.8619],
         [ 0.7827,  0.3786, -0.4940]]])

# Test TangentSpaceGaussian log_prob_from_params

In [18]:
torch.eye(3).repeat(2,1,1).size()

torch.Size([2, 3, 3])

In [19]:
torch.ones(3).repeat(2,1).size()

torch.Size([2, 3])

In [20]:
tsg.log_prob_from_params(torch.eye(3).repeat(2,1,1), torch.ones(3))

self.sigma size:  torch.Size([3])
omega size:  torch.Size([3])
actions size:  torch.Size([2, 3, 3])
actions type:  <class 'torch.Tensor'>
self.sigma size:  torch.Size([3])
omega size:  torch.Size([3])
actions:  torch.Size([2, 3, 3])
mu:  torch.Size([2, 3, 3])
sigma:  torch.Size([3])


(tensor([[[ 0.4112,  0.8225,  0.3930],
          [-0.7022,  0.5607, -0.4388],
          [-0.5813, -0.0955,  0.8081]],
 
         [[ 0.4112,  0.8225,  0.3930],
          [-0.7022,  0.5607, -0.4388],
          [-0.5813, -0.0955,  0.8081]]]),
 tensor([-1.3875, -1.3875]))

In [21]:
x = torch.randn(2, 4, 4)
y = torch.linalg.inv(x)
y

tensor([[[ 0.2897, -0.5002,  0.1180, -0.0212],
         [-0.0530, -0.4179,  0.9735,  1.7518],
         [-0.1357, -0.2728,  0.1096, -0.0746],
         [ 0.1607,  0.2180,  0.0097, -0.7048]],

        [[ 0.5245, -0.1162,  0.2799,  0.3889],
         [ 0.2064, -0.4569,  0.0027,  0.7802],
         [-0.6557, -0.2177,  0.4619,  1.0736],
         [ 0.3097, -0.1232,  0.0108,  1.1752]]])

Again, codes can run, but need to check correctness.

# Try to run training

In [22]:
import torch
from absl import app, flags
from stable_baselines3 import SAC, PPO
from envs.wahba import Wahba
from stable_baselines_utils import CustomSACPolicy, \
    CustomCNN

In [23]:
def main(argv):
    env = Wahba()
    device = torch.device('cpu')
    policy_kwargs = dict(
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = dict(features_dim = 256))
    policy_kwargs['n_critics'] = 1
    policy_kwargs['share_features_extractor'] = False
    policy = CustomSACPolicy
    model = SAC(policy, env, verbose = 1, ent_coef = 'auto_0.1',
                policy_kwargs = policy_kwargs, device = device)
    model.learn(total_timesteps = 110, eval_freq = 5, n_eval_episodes = 5)

In [24]:
main(None)

/home/fantasticoven/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: not enough values to unpack (expected 4, got 3)

# Experiments for batch operations

In [ ]:
sigma = torch.ones(1, 3)
omiga = torch.normal(torch.zeros(1, 3), sigma)
omiga

In [ ]:
def transfer(omiga):
    omiga_0, omiga_1, omiga_2 = omiga[0], omiga[1], omiga[2]
    omiga_hat = torch.tensor([[0, -omiga_2, omiga_1],
                                [omiga_2, 0, -omiga_0],
                                [-omiga_1, omiga_0, 0]])
    return omiga_hat

In [ ]:
from functorch import vmap
batch_transfer = vmap(transfer)
batch_transfer(omiga)

In [ ]:
from liegroups.torch import SO3
C = SO3.exp(torch.Tensor([[1,2,3],
                          [0,0,0]]))
print(torch.Tensor([[1,2,3],
                          [0,0,0]]).size())
SO3.log(C)

In [ ]:
np.log(1)

# Question to ask: the original wahba problem action is (4,), in our case actions are (3,3).